In [ ]:
!unzip /content/data.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import glob
import shutil

In [ ]:
# number of images in directory
root_dir="/content/Brain Tumor Data Set/Brain Tumor Data Set"
no_of_images={}
for dir in os.listdir(root_dir):
  # no_of_images[dir]=os.listdir(os.path.join(root_dir,dir))
  no_of_images[dir]=len(os.listdir(os.path.join(root_dir,dir)))
no_of_images.items()

In [ ]:
os.listdir("/content/Brain Tumor Data Set/Brain Tumor Data Set")

In [ ]:
len(os.listdir("/content/Brain Tumor Data Set/Brain Tumor Data Set"))

#@ split data
##70% training data
## 15% validation data
## 15% testing data

In [ ]:
# train folder
def datafolder(p,split):
  if not os.path.exists("./"+p):
    os.mkdir("./"+p)
    for dir in os.listdir(root_dir):
      os.makedirs("./"+p+"/"+dir)
      for img in np.random.choice(a=os.listdir(os.path.join(root_dir,dir)),
                                  size=(math.floor(split*no_of_images[dir])-2),
                                  replace=False):
        o=os.path.join(root_dir,dir,img)
        d=os.path.join("./"+p,dir)
        shutil.copy(o,d)
        os.remove(o)
  else:
    print(f"{p}folder exists")

In [ ]:
datafolder("train",0.7)

In [ ]:
datafolder("val",0.15)

In [ ]:
datafolder("test",0.15)

### model build

In [ ]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential,Model,load_model
from keras.preprocessing.image import load_img,img_to_array
from keras.applications.mobilenet import MobileNet,preprocess_input
import keras

In [ ]:
basemodel=MobileNet(input_shape=(224,224,3),include_top=False)

In [ ]:
for layer in basemodel.layers:
  layer.trainable=False

In [ ]:
basemodel.summary()

In [ ]:
from keras.backend import flatten
x=Flatten()(basemodel.output)
x=Dense(units=1,activation='sigmoid')(x)
model=Model(basemodel.input,x)

In [ ]:
# CNN Model
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(224,224,3)))
model.add(Conv2D(filters=36,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

###preparing data using data generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def preprocessingImages(path):
  """
  input : path
  output: preprocessed image
  """
  image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255,horizontal_flip=True)
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path="/content/train"
train_data=preprocessingImages(path)

In [ ]:
def preprocessingImages2(path):
  """
  input : path
  output: preprocessed image
  """
  image_data=ImageDataGenerator(preprocessing_function=preprocess_input)
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path="/content/test"
test_data=preprocessingImages2(path)

In [ ]:
path="/content/val"
val_data=preprocessingImages2(path)

In [ ]:
# early stopping
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1,mode='auto')
# model checkpoint
mcheck=ModelCheckpoint(monitor='val_accuracy',filepath='./bestmodel.h5',verbose=1,save_best_only=True,mode='auto')
cd=[es,mcheck]

In [ ]:
from keras import callbacks
# model training
hs=model.fit_generator(generator=train_data,steps_per_epoch=8,epochs=30,verbose=1,validation_data=val_data,validation_steps=16,callbacks=cd)


In [ ]:
h=hs.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'])
plt.title('accuracy VS Validation-accuracy')
plt.show()

In [ ]:
from keras.models import load_model
model=load_model("/content/bestmodel.h5")

In [ ]:
# Accuracy
acc=model.evaluate(test_data)[1]
print("Accuracy is :" ,acc*100,"%")

In [ ]:
from keras.preprocessing.image import load_img,img_to_array


In [ ]:
test_data.class_indices

In [ ]:
def detect(path):
  print("Welcome To BrainCancer Detection Program")
  img=load_img(path,target_size=(224,224))
  img=img_to_array(img)/255
  
  plt.imshow(img)
  plt.show()
  img=np.expand_dims(img,axis=0)
  pred=model.predict(img)
  if pred>0.5:
    print(pred)
    print("it has no Brain tumor")
  else:
    print(pred)
    print("it has Brain tumor")
detect("/content/test/Brain Tumor/Cancer (1107).jpg")

In [ ]:
detect("/content/test/Brain Tumor/Cancer (1116).jpg")

In [ ]:
detect("/content/test/Healthy/Not Cancer  (1010).jpg")

In [ ]:
detect("/content/test/Healthy/Not Cancer  (1755).jpg")

In [ ]:
detect("/content/test/Healthy/Not Cancer  (1798).jpg")